In [1]:
from IPython.display import display
import os
os.chdir('./../')
exp = './experiment_real/'
from experiment_real.dataset import Dictionary, VQAFeatureDataset
dictionary = Dictionary.load_from_file(exp+'data/dictionary.pkl')
train_dset = VQAFeatureDataset('val', dictionary, dataroot=exp+'data/')
import pandas as pd
os.chdir('./visualization/')
import numpy as np
pd.options.display.max_rows = None
import json
from collections import Counter

loading dictionary from ./experiment_real/data/dictionary.pkl
loading features from h5 file


In [2]:
with open('./../experiment_real/data/v2_mscoco_val2014_annotations.json') as file:
    original_questions = json.load(file)

In [3]:
original_questions['annotations'].sort(key= lambda x: x['question_id'])

In [4]:
len(original_questions['annotations'])

214354

In [5]:
types = [q['question_type'] for q in original_questions['annotations']]
count_types = Counter(types)
check_type_order = sorted(list(set(count_types)))[::-1]
questions = [q['question'] for q in train_dset.entries]

outliers = []
for q in questions:
    add = True
    for t in check_type_order:
        if t in q.lower():
            add = False
            break
    if add:
        outliers.append(q.lower())
        
new_types = Counter([' '.join(w.strip().split()[:2]).strip() for w in outliers]).keys()
count_types.update(new_types)
check_type_order = sorted(list(set(count_types)))[::-1]

questions_types = []
for q in questions:
    add = True
    for t in check_type_order:
        if t in q.replace('  ',' ').lower():
            questions_types.append(t)
            add = False
            break
    if add:
        print(q)
len(questions_types)

214354

In [6]:
data = pd.read_csv('./data/responses_real_val', sep='\t', header=None)

In [7]:
data.shape

(214354, 2048)

In [8]:
data['q_type'] = questions_types

In [9]:
data.to_csv('data/real_q_types_features_val.csv', header=None, index=None)